In [ ]:
"""
Calculate Actual Growth Rates from Feature Table
------------------------------------------------
Author: [Your Name]
Date: [Insert Date]

Purpose:
This script calculates the actual change in OTU abundance between consecutive samples
from a QIIME2 feature table (`feature_table.qza`). The column headers in the output 
represent the epoch time of the first day in each pair of consecutive samples, aligning 
with MICOM inputs for growth rate prediction.

Workflow:
1. Export the feature table from QIIME2 (`.qza`) to `.biom` format.
2. Convert the `.biom` file to a readable `.tsv` file.
3. Load the `.tsv` into a pandas DataFrame and sort columns (samples) by epoch time.
4. Calculate the change in abundance (ΔAbundance) for each OTU between consecutive days.
5. Save the resulting growth rates as a `.csv` file.

Inputs:
- A QIIME2 feature table (`feature_table.qza`) with OTU abundances.
- Sample IDs (column headers) as epoch time values in seconds.

Outputs:
- `actual_growth_rates.csv`: A table with OTU IDs as rows and growth rates between 
  consecutive days as columns (column headers = epoch time of the first day).

Dependencies:
- QIIME2 must be installed and accessible in the environment.
- BIOM CLI must be installed for file format conversions.
- Python libraries: pandas, numpy, subprocess, pathlib.

Usage:
    python calculate_actual_growth_rates.py \
        --feature_table_qza <path_to_feature_table.qza> \
        --output_dir <output_directory>

Example:
    python calculate_actual_growth_rates.py \
        --feature_table_qza ../data/qiime_outputs/F01_feature_table.qza \
        --output_dir ../data/actual_growth_rates/
"""



In [ ]:
import pandas as pd
import numpy as np
import subprocess
from pathlib import Path
import argparse

def main(feature_table_qza, output_dir):
    """
    Main function to calculate actual growth rates from a feature table.
    
    Parameters:
        - feature_table_qza (str): Path to the input QIIME2 feature table (.qza).
        - output_dir (str): Path to the directory where outputs will be saved.
    """
    # Ensure the output directory exists
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Step 1: Export the feature table from QIIME2 artifact to .biom format
    print("Exporting feature table from QIIME2 artifact...")
    export_dir = Path(output_dir) / "qiime_exports"
    export_dir.mkdir(parents=True, exist_ok=True)

    subprocess.run([
        "qiime", "tools", "export",
        "--input-path", feature_table_qza,
        "--output-path", str(export_dir)
    ], check=True)

    feature_table_biom = export_dir / "feature-table.biom"

    # Step 2: Convert the .biom file to a .tsv file
    print("Converting feature table to TSV format...")
    feature_table_tsv = export_dir / "feature-table.tsv"

    subprocess.run([
        "biom", "convert",
        "-i", str(feature_table_biom),
        "-o", str(feature_table_tsv),
        "--to-tsv"
    ], check=True)

    # Step 3: Load the feature table into a pandas DataFrame
    print("Loading feature table...")
    feature_table = pd.read_csv(feature_table_tsv, sep="\t", skiprows=1, index_col=0)

    # Step 4: Sort samples (columns) by epoch time
    print("Sorting samples by epoch time...")
    sorted_columns = sorted([int(col) for col in feature_table.columns])  # Convert column names to integers for sorting
    feature_table = feature_table[sorted(map(str, sorted_columns))]  # Reorder columns by sorted epoch time

    # Step 5: Calculate daily changes in abundance
    print("Calculating actual growth rates...")
    otu_ids = feature_table.index
    actual_growth_rates = {}

    for i in range(len(sorted_columns) - 1):
        day1 = str(sorted_columns[i])      # Current day's epoch time
        day2 = str(sorted_columns[i + 1])  # Next day's epoch time

        # Calculate ΔAbundance for all OTUs between day1 and day2
        actual_growth_rates[day1] = feature_table[day2] - feature_table[day1]

    # Convert results into a DataFrame
    growth_rate_df = pd.DataFrame(actual_growth_rates)

    # Step 6: Save the actual growth rates to a CSV file
    output_path = Path(output_dir) / "actual_growth_rates.csv"
    growth_rate_df.to_csv(output_path)

    print(f"Actual growth rates saved to: {output_path}")


if __name__ == "__main__":
    # Set up argument parser
    parser = argparse.ArgumentParser(description="Calculate actual growth rates from a QIIME2 feature table.")
    parser.add_argument(
        "--feature_table_qza", required=True, 
        help="Path to the input QIIME2 feature table (.qza)."
    )
    parser.add_argument(
        "--output_dir", required=True, 
        help="Path to the directory where outputs will be saved."
    )

    # Parse arguments
    args = parser.parse_args()

    # Run the main function
    main(
        feature_table_qza=args.feature_table_qza,
        output_dir=args.output_dir
    )

In [1]:
!python ../scripts/calculate_actual_growth_rates.py \
    --feature_tables_dir ../data/qiime_outputs/ \
    --output_dir ../data/actual_growth_rates/

Exporting feature table for subject M01...
Exported ../data/qiime_outputs/M01_feature_table.qza as BIOMV210DirFmt to directory ../data/actual_growth_rates/qiime_exports/M01
Converting feature table for subject M01 to TSV format...
Loading feature table for subject M01...
Sorting samples by epoch time for subject M01...
Calculating actual growth rates for subject M01...
Actual growth rates saved for subject M01 at: ../data/actual_growth_rates/actual_growth_rates_M01.csv
Exporting feature table for subject M02...
Exported ../data/qiime_outputs/M02_feature_table.qza as BIOMV210DirFmt to directory ../data/actual_growth_rates/qiime_exports/M02
Converting feature table for subject M02 to TSV format...
Loading feature table for subject M02...
Sorting samples by epoch time for subject M02...
Calculating actual growth rates for subject M02...
Actual growth rates saved for subject M02 at: ../data/actual_growth_rates/actual_growth_rates_M02.csv
Exporting feature table for subject F01...
Exported 

## calculate_actual_growth_rates_genus.py
added taxonomy.qza of genus level to collapse actual growth rates by genus


In [1]:
!python ../scripts/calculate_actual_growth_rates_genus.py \
    --feature_tables_dir ../data/qiime_outputs/ \
    --taxonomy_dir ../data/qiime_outputs \
    --output_dir ../data/actual_growth_rates_genus/

Exported ../data/qiime_outputs/M01_taxonomy.qza as TSVTaxonomyDirectoryFormat to directory ../data/actual_growth_rates_genus/taxonomy_exports
Exporting feature table for subject M01...
Exported ../data/qiime_outputs/M01_feature_table.qza as BIOMV210DirFmt to directory ../data/actual_growth_rates_genus/qiime_exports/M01
Converting feature table for subject M01 to TSV format...
Loading feature table for subject M01...
Sorting samples by epoch time for subject M01...
Calculating actual growth rates for subject M01...
Collapsing growth rates by genus for subject M01...
Actual growth rates saved for subject M01 at: ../data/actual_growth_rates_genus/M01_actual_growth_rates_by_genus.csv
Exported ../data/qiime_outputs/M02_taxonomy.qza as TSVTaxonomyDirectoryFormat to directory ../data/actual_growth_rates_genus/taxonomy_exports
Exporting feature table for subject M02...
Exported ../data/qiime_outputs/M02_feature_table.qza as BIOMV210DirFmt to directory ../data/actual_growth_rates_genus/qiime_ex

In [11]:
!python3 ../scripts/calculate_actual_growth_rates_genus.py \
    --feature_tables_dir ../data/qiime_outputs/ \
    --taxonomy_dir ../data/qiime_outputs \
    --output_dir ../data/actual_growth_rates_genus_2/

Exported ../data/qiime_outputs/M01_taxonomy.qza as TSVTaxonomyDirectoryFormat to directory ../data/actual_growth_rates_genus_2/taxonomy_exports
Exporting feature table for subject M01...
Exported ../data/qiime_outputs/M01_feature_table.qza as BIOMV210DirFmt to directory ../data/actual_growth_rates_genus_2/qiime_exports/M01
Converting feature table for subject M01 to TSV format...
Loading feature table for subject M01...
Sorting samples by epoch time for subject M01...
Normalizing feature table for subject M01 to relative abundances...
Sum of each sample BEFORE normalization:
 1200528000    7500.0
1200614400    7500.0
1200700800    7500.0
1200787200    7500.0
1200873600    7500.0
               ...  
1208736000    7500.0
1208822400    7500.0
1208908800    7500.0
1208995200    7500.0
1209081600    7500.0
Length: 100, dtype: float64
Collapsing growth rates by genus for subject M01...
Actual growth rates saved for subject M01 at: ../data/actual_growth_rates_genus_2/M01_actual_growth_rates_